## QIR<>OQ testing

Generate QIR instructions using PyQIR

In [1]:
from pyqir.generator import BasicQisBuilder, SimpleModule, ir_to_bitcode
from itertools import combinations
from sympy import Symbol

In [19]:
from pyqir.parser import PyQirParameter

ModuleNotFoundError: No module named 'pyqir.parser.pyqir'

In [2]:
qaoa_module = SimpleModule("QAOA", num_qubits=6, num_results=6)
qis = BasicQisBuilder(qaoa_module.builder)

In [3]:
#construct the circuit
gamma = 1#Symbol('\gamma')
beta = 1#Symbol('\beta')

for i in qaoa_module.qubits:
    qis.h(i)

for i,j in list(combinations(qaoa_module.qubits,2)):
    qis.cx(i,j)
    qis.rz(gamma, j)
    qis.cx(i,j)
    
for k in qaoa_module.qubits:
    qis.rx(beta, k)
    
for idx,mq in enumerate(qaoa_module.qubits):
    qis.m(mq, qaoa_module.results[idx])

In [4]:
ir = qaoa_module.ir()

In [12]:
from openqaoa.problems.problem import MaximumCut
from openqaoa.workflows.optimizer import QAOA
from openqaoa.devices import create_device

In [8]:
prob = MaximumCut.random_instance(n_nodes=7, edge_probability=0.5)
prob_qubo = prob.get_qubo_problem()

In [13]:
dev = create_device('local','qiskit.qasm_simulator')
q = QAOA()
q.set_device(dev)
q.compile(prob_qubo)

In [17]:
ckt = q.backend.parametric_circuit